In [ ]:
# create a seperate folder to store everything
!mkdir finetuning
%cd finetuning

In [ ]:
# clone the repo for running finetuning
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

In [ ]:
ls

In [ ]:
! sudo apt install tree

# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install wandb
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install --editable ./
%cd ..

In [ ]:
import wandb
# This is secret and shouldn't be checked into version control
WANDB_API_KEY='c32fbee4e85afebf8006826b6f255e9421a35085'
# Name and notes optional

wandb.login(key=WANDB_API_KEY)
wandb.init(project="kn-te", entity="adityavyawahare")
# # login authorization.

In [ ]:
ls


In [ ]:
! pip install -U --no-cache-dir gdown --pre

In [ ]:
%cd ..
!mkdir drive
%cd drive
!mkdir MyDrive
%cd MyDrive
!mkdir Dravidian_Dataset_Full
%cd Dravidian_Dataset_Full

In [ ]:
!gdown --id 12RI9PzWR0c3fR83j2f2VG5yhwvDKMKrf --folder

In [ ]:
!gdown --id 11o3czc-T8sVr_h_2YxQOroaOnWcFzZ-9--folder
# !gdown --id 1Pz3oFKy6VpQVL0Bbi3B0mCUZOIkIIXeC --folder

In [ ]:
!gdown --id 12JMZrf35C8Uc9uWgqGsrAFp2CI6EvcU9 --folder #backtranslation

In [ ]:
!gdown --id 109_Hl10D9x7TcprsR5cV3MTuKNkDYH0A --folder

In [ ]:
ls

In [ ]:
%cd ../../../finetuning/indicTrans

In [ ]:
%env JOBLIB_TEMP_FOLDER=/tmp

In [ ]:
ls

In [ ]:
# Finetuning the model

# pls refer to fairseq documentaion to know more about each of these options (https://fairseq.readthedocs.io/en/latest/command_line_tools.html)


# some notable args:
# --max-update=1000     -> for this example, to demonstrate how to finetune we are only training for 1000 steps. You should increase this when finetuning
# --arch=transformer_4x -> we use a custom transformer model and name it transformer_4x (4 times the parameter size of transformer  base)
# --user_dir            -> we define the custom transformer arch in model_configs folder and pass it as an argument to user_dir for fairseq to register this architechture
# --lr                  -> learning rate. From our limited experiments, we find that lower learning rates like 3e-5 works best for finetuning.
# --restore-file        -> reload the pretrained checkpoint and start training from here (change this path for indic-en. Currently its is set to en-indic)
# --reset-*             -> reset and not use lr scheduler, dataloader, optimizer etc of the older checkpoint
# --max_tokns           -> this is max tokens per batch


!( fairseq-train ../../drive/MyDrive/Dravidian_Dataset_Full/mono_final_bin \
--max-source-positions=1320 \
--max-target-positions=1320 \
--max-epoch=3 \
--share-all-embeddings \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--warmup-updates 1000 \
--dropout 0.2 \
--lr-scheduler=inverse_sqrt \
--lr 3e-5 \
--tensorboard-logdir ../../../tmp/tensorboard-wandb \
--save-dir ../../drive/MyDrive/Dravidian_Dataset_Full/model \
--skip-invalid-size-inputs-valid-test \
--fp16 \
--user-dir model_configs \
--update-freq=2 \
--distributed-world-size 1 \
--max-tokens 1568 \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
--eval-bleu-detok moses \
--eval-bleu-remove-bpe \
--eval-bleu-print-samples \
--best-checkpoint-metric bleu \
--maximize-best-checkpoint-metric \
--no-epoch-checkpoints \
--restore-file ../../drive/MyDrive/Dravidian_Dataset_Full/model/checkpoint_best.pt \
--wandb-project "kn-ta backtranslation_strip final pls" \
--reset-lr-scheduler \
--reset-meters \
--reset-dataloader \
--reset-optimizer)

In [ ]:
# Finetuning the model

# pls refer to fairseq documentaion to know more about each of these options (https://fairseq.readthedocs.io/en/latest/command_line_tools.html)


# some notable args:
# --max-update=1000     -> for this example, to demonstrate how to finetune we are only training for 1000 steps. You should increase this when finetuning
# --arch=transformer_4x -> we use a custom transformer model and name it transformer_4x (4 times the parameter size of transformer  base)
# --user_dir            -> we define the custom transformer arch in model_configs folder and pass it as an argument to user_dir for fairseq to register this architechture
# --lr                  -> learning rate. From our limited experiments, we find that lower learning rates like 3e-5 works best for finetuning.
# --restore-file        -> reload the pretrained checkpoint and start training from here (change this path for indic-en. Currently its is set to en-indic)
# --reset-*             -> reset and not use lr scheduler, dataloader, optimizer etc of the older checkpoint
# --max_tokns           -> this is max tokens per batch


!( fairseq-train ../../drive/MyDrive/Dravidian_Dataset_Full/final_bin \
--max-source-positions=1320 \
--max-target-positions=1320 \
--max-epoch=6 \
--share-all-embeddings \
--arch=transformer_4x \
--criterion=label_smoothed_cross_entropy \
--source-lang=SRC \
--target-lang=TGT \
--label-smoothing=0.1 \
--optimizer adam \
--adam-betas "(0.9, 0.98)" \
--clip-norm 1.0 \
--warmup-init-lr 1e-07 \
--warmup-updates 1000 \
--dropout 0.2 \
--lr-scheduler=inverse_sqrt \
--lr 3e-5 \
--tensorboard-logdir ../../../tmp/tensorboard-wandb \
--save-dir ../../drive/MyDrive/Dravidian_Dataset_Full/model \
--skip-invalid-size-inputs-valid-test \
--fp16 \
--user-dir model_configs \
--update-freq=2 \
--distributed-world-size 1 \
--max-tokens 1568 \
--eval-bleu \
--eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
--eval-bleu-detok moses \
--eval-bleu-remove-bpe \
--eval-bleu-print-samples \
--best-checkpoint-metric bleu \
--maximize-best-checkpoint-metric \
--no-epoch-checkpoints \
--restore-file ../../drive/MyDrive/Dravidian_Dataset_Full/model/checkpoint_best.pt \
--wandb-project "kn-ml backtranslation_strip final pls" \
--reset-lr-scheduler \
--reset-meters \
--reset-dataloader \
--reset-optimizer)

In [ ]:
%cd ..
!mkdir data
%cd data

In [ ]:
!gdown --id 1P7ZTatFxoLxjr80fE2ogO7aj0xKdesij --folder

In [ ]:
%cd ..
%cd indicTrans

In [ ]:

# To test the models after training, you can use joint_translate.sh
# joint_translate takes src_file, output_fname, src_lang, tgt_lang, model_folder as inputs
# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
#              as, bn, hi, gu, kn, ml, mr, or, pa, ta, te)
# supported languages are:
#              as - assamese, bn - bengali, gu - gujarathi, hi - hindi, kn - kannada, 
#              ml - malayalam, mr - marathi, or - oriya, pa - punjabi, ta - tamil, te - telugu

# model_dir -> the directory containing the model and the vocab files

# Note: if the translation is taking a lot of time, please tune the buffer_size and batch_size parameter for fairseq-interactive defined inside this joint_translate script
# exp_dir= ../data/kn-ml/dev/dev.kn
# download_dir= ../m2m

# here we are translating the english sentences to hindi
!bash joint_translate.sh ../data/kn-ta/dev/dev.kn ../data/kn-ta/kn_ta_outputs.txt 'kn' 'ta' ../../drive/MyDrive/Dravidian_Dataset_Full

In [ ]:
!cat ../data/kn-ta/kn_ta_outputs.txt

In [ ]:
ls

In [ ]:
# to compute bleu scores for the predicitions with a reference file, use the following command
# arguments:
# pred_fname: file that contains model predictions
# ref_fname: file that contains references
# src_lang and tgt_lang : the source and target language

!bash compute_bleu.sh ../data/kn-ta/kn_ta_outputs.txt ../data/kn-ta/dev/dev.ta 'kn' 'ta'

In [ ]:
# ! fairseq-generate ../data/kn-ml \
# --path ../../drive/MyDrive/Dravidian_Dataset_Full/checkpoint1.pt \
# --batch-size 64 \
# --beam 5 \
# --seed 1 \
# --scoring bleu \
# --wandb-project "kn-ml fairseq"

In [ ]:
predicted = open("../data/kn-ta/kn_ta_outputs.txt")
actual= open("../data/kn-ta/dev/dev.ta")
p = predicted.readlines()
a = actual.readlines()
count=0
lmt=10
for x,y in zip(p,a):
    print('----------------------------------------------',count+1,'-----------------------------------')
    print('A: ',y)
    print('P: ',x)
    count+=1
    if count==lmt:
        break

In [ ]:
# cat ../data/kn-ml/kn_ml_outputs.txt

In [ ]:
from indicnlp.tokenize.indic_tokenize import trivial_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import sentence_tokenize
import re

INDIC = ["as", "bn", "gu", "hi", "kn", "ml", "mr", "or", "pa", "ta", "te"]

def split_sentences(paragraph, language):
    pattern=re.compile(r'\s*[.]\s*')
    matches=pattern.split(paragraph)
    final=[]
    for match in matches:
        if len(match)==0:
            continue
        if len(match)>200:
            m=re.compile(r'\s*[,]\s*').split(paragraph)
            lst=[]
            s=""
            for x in m:
                x+=", "
                if x=="":
                    continue
                if len(s+x)<200:
                    s+=x
                else:
                    lst.append(s)
                    s=x
            lst.append(s[:-2])
            final.extend(lst)
        else:
            final.append(match)
    return final


lang = 'kn'
input_path = '../data/kn-ta/dev/dev.kn'
output_path = '../data/kn-ta/kn_strip.txt'

count=-1
dic={}
with open(input_path, 'r', encoding='utf-8') as in_fp,\
	 open(output_path, 'w', encoding='utf-8') as out_fp:
    for line in in_fp.readlines():
        count+=1
        sent = line.rstrip('\n')
        if len(sent)>200:
            sent_list=split_sentences(sent,'kn')
            dic[str(count)]=len(sent_list)
            for lol in sent_list:
                out_fp.write(lol)
                out_fp.write('\n')
        else:
            out_fp.write(sent)
            out_fp.write('\n')
print(dic)

In [ ]:
!bash joint_translate.sh ../data/kn-ta/kn_strip.txt ../data/kn-ta/kn_ta_strip_outputs.txt 'kn' 'ta' ../../drive/MyDrive/Dravidian_Dataset_Full

In [ ]:
from indicnlp.tokenize.indic_tokenize import trivial_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import sentence_tokenize
import re

lang = 'ta'
output_path = '../data/kn-ta/ta_combined.txt'
input_path = '../data/kn-ta/kn_ta_strip_outputs.txt'

count=-1 
with open(input_path, 'r', encoding='utf-8') as in_fp,\
	 open(output_path, 'w', encoding='utf-8') as out_fp:
    line=in_fp.readlines()
    i=0
    while i < len(line):
        if str(i) in dic.keys():
            for j in range(dic[str(i)]):
                sent = line[i].rstrip('\n')
                out_fp.write(sent)
                out_fp.write(" ")
                i+=1
            out_fp.write('\n')
        else:
            sent = line[i].rstrip('\n')
            out_fp.write(sent)
            out_fp.write('\n')
            i+=1

In [ ]:
file = open("../data/kn-ta/ta_combined.txt","r")
Counter = 0
  
# Reading from file
Content = file.read()
CoList = Content.split("\n")
  
for i in CoList:
    if i:
        Counter += 1
print("This is the number of lines in the file")
print(Counter)

In [ ]:
!bash compute_bleu.sh ../data/kn-ta/ta_combined.txt ../data/kn-ta/dev/dev.ta 'kn' 'ta'

In [ ]:
# !cat ../data/kn-ml/kn_ml_strip_outputs.txt

In [ ]:
# cat ../data/kn-ml/ml_combined.txt

In [ ]:
from indicnlp.tokenize.indic_tokenize import trivial_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import sentence_tokenize
import re

INDIC = ["as", "bn", "gu", "hi", "kn", "ml", "mr", "or", "pa", "ta", "te"]

def split_sentences(paragraph, language):
    pattern=re.compile(r'\s*[.]\s*')
    matches=pattern.split(paragraph)
    final=[]
    for match in matches:
        if len(match)==0:
            continue
        if len(match)>200:
            m=re.compile(r'\s*[,]\s*').split(paragraph)
            lst=[]
            s=""
            for x in m:
                x+=", "
                if x=="":
                    continue
                if len(s+x)<200:
                    s+=x
                else:
                    lst.append(s)
                    s=x
            lst.append(s[:-2])
            final.extend(lst)
        else:
            final.append(match)
    return final


lang = 'kn'
input_path = '../data/kn-ta/test/test.kn'
output_path = '../data/kn-ta/test_strip.txt'

count=-1
dic={}
with open(input_path, 'r', encoding='utf-8') as in_fp,\
	 open(output_path, 'w', encoding='utf-8') as out_fp:
    for line in in_fp.readlines():
        count+=1
        sent = line.rstrip('\n')
        if len(sent)>200:
            sent_list=split_sentences(sent,'kn')
            dic[str(count)]=len(sent_list)
            for lol in sent_list:
                out_fp.write(lol)
                out_fp.write('\n')
        else:
            out_fp.write(sent)
            out_fp.write('\n')
print(dic)

In [ ]:
!bash joint_translate.sh ../data/kn-ta/test_strip.txt ../data/kn-ta/test_strip_outputs.txt 'kn' 'ta' ../../drive/MyDrive/Dravidian_Dataset_Full

In [ ]:
from indicnlp.tokenize.indic_tokenize import trivial_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import sentence_tokenize
import re

lang = 'ta'
output_path = '../data/kn-ta/ta_test_pred.txt'
input_path = '../data/kn-ta/test_strip_outputs.txt'

count=-1 
with open(input_path, 'r', encoding='utf-8') as in_fp,\
	 open(output_path, 'w', encoding='utf-8') as out_fp:
    line=in_fp.readlines()
    i=0
    while i < len(line):
        if str(i) in dic.keys():
            for j in range(dic[str(i)]):
                sent = line[i].rstrip('\n')
                out_fp.write(sent)
                out_fp.write(" ")
                i+=1
            out_fp.write('\n')
        else:
            sent = line[i].rstrip('\n')
            out_fp.write(sent)
            out_fp.write('\n')
            i+=1

In [ ]:
cat ../data/kn-ta/ta_test_pred.txt